# Part 12: Prop, Backprop
Let's review.

We saw that an untrained network predicts about 50% for everything, since it has seen no examples. Then we gave it a few examples and it learned a little bit. Then we gave it more, and it learned more.

Then it got stuck, and we used some tricks to perturb the few samples we had to make a lot more, and suddenly the learning got better and with fewer epochs.

Then we gave it more colors and even more, and it worked pretty well. We had so many perturbed color samples that we were able to keep some examples to test against and see if the network was doing a good job of learning more than just what it was given. It did.

But how did it work? How was it able to learn at all with all those examples?

## Give Me Some Bias
A plain-vanilla untrained network has some small numbers for its biases and weights, but really you can think of it as a blank slate. What our training does is provide feedback to the network to tell it to make itself better. We give it an example by setting the R, G, B sensors to the values of a color like gray, (0.5, 0.5, 0.5), then letting the network run the math forward through the network. This is *forward propagation*. The RGB values are received at the middle layer perceptrons, multiplied by the weights, added together, and if the value is big enough, the perceptrons get triggered and pass on a value to the output perceptrons. (Remember, this is just like the the perceptrons we played with in lesson 8 - they add their inputs and maybe become triggered.) Those perceptrons have weights of their own, multiplying the inputs from the middle layer, adding the results together, and we get the predictions as floating point numbers, where the number is between zero and 1 and is the network's decision about the probabilty of each of the colors being a match for the inputs.

![ColorMatchingNetDiagram.png](./ColorMatchingNetDiagram.png)

## Make a Mistake, Walk It Back
We kind of knew all that already. And when we get the resulting outputs, they start out right around 50% for each of the colors, which means uncertain. But how do we tell the network to get better?

This is where *backpropagation* comes in. Just as we stepped *forward* through the network, starting at the incoming color values and applying our perceptron math operations at each step, so we compare the result against what we wanted (this is called the *error*), walk *back* to each perceptron in the middle layer and adjust the biases and weights to get it closer to calculating the right result. This adjustment is called *backpropagation*. So that during the next epoch the weights and biases produce a better result.

## Batch It
When our batch size hyperparameter is 1, we do all of this work for each color sample: We show the color sample to the RGB sensors, run the math forward through the network, find how far away from the right color prediction the network was, then *backpropagate* through the network, adjusting things a bit, and on and on. And then do it all again for the next epoch, and again and again.

When the batch size is more than 1, we run several samples through in the forward direction, tracking the error for each one, then run backpropagation once to get the weights closer for all of the samples at once. This is why a bigger batch size makes for less accurate training - we are combining the adjustments for a single backpropagation, which blends the adjustments together, which is less precise than doing it each time. But backpropagation is expensive in terms of math, and we are impatient, so we try different batch sizes to see what takes less time but is good enough.

## Show Me
This is all fine, but it's better if we learn by seeing and doing, not just talking and thinking. Lets go back to the beginning and look at our earliest network, but this time let's take a look inside the math. Below we create a program that lets us look inside a small network, and we'll feed it three samples, black, gray, and white, and see what happens.

Note that we're only animating a batch size of 1. And you will be manualy running the epochs by selecting which color to train on for each iteration.

First, some helper code:

In [3]:
from ipywidgets import interact
import time, random

def centerText(displayWidth, text, c=' '):
    pre = (displayWidth - len(text)) // 2
    post = displayWidth - len(text) - pre
    return c * pre + text + c * post

def formatArray(displayWidth, arr):
    return centerText(displayWidth, " ".join([ ("% .2f" % a) for a in arr ]))

def red(text):
    return "\033[31m" + text + "\033[0m"  # ANSI console color code

Next, a Perceptron class so the code related to a perceptron is in one place.

In [16]:
class Perceptron:
    def __init__(self, numInputs, learningRate = 0.5):
        self.bias = 0.0
        self.weights = [random.gauss(0.0, 0.01) for i in range(numInputs)]
        self.currentInputs = [0.0] * numInputs
        self.learningRate = learningRate
        self.sumOfWeightScaledInputs = 0.0
        self.output = 0.0
        self.triggered = False
        self.printWidth = 5 * numInputs + (numInputs - 1)
        self.error = 0.0
    
    def display(self, showInputs):
        if showInputs:
            print(self.getInputsDisplay())
        print(self.getWeightsDisplay())
        print(self.getBiasDisplay())

    def getInputsDisplay(self):
        return formatArray(self.printWidth, self.currentInputs)
    
    def getWeightsDisplay(self):
        return formatArray(self.printWidth, self.weights)
    
    def getValueDisplay(self, value):
        return centerText(self.printWidth, "% .2f" % value)
        
    def getBiasDisplay(self):
        return self.getValueDisplay(self.bias)

    def getBiasCheckDisplay(self):
        return centerText(self.printWidth, "% .2f > % .2f ?" % (self.sumOfWeightScaledInputs, self.bias))

    def getOutputDisplay(self):
        return self.getValueDisplay(self.output)
    
    def acceptInputsCheckIfTriggered(self, inputs):
        # inputs is an array of size numInputs.
        self.currentInputs = inputs
        self.sumOfWeightScaledInputs = 0
        for i in range(len(inputs)):
            self.sumOfWeightScaledInputs += inputs[i] * self.weights[i]
        self.triggered = (self.sumOfWeightScaledInputs > self.bias)
        if (self.triggered):
            self.output = 1.0
        else:
            self.output = 0.0

    def backPropagate(self, expectedOutput):
        # expectedOutput is the value the next layer was expecting to see. If this
        # is an output neuron, it's 1 if this neuron should have predicted the related
        # item at 100%, or 0 if it should have predicted 0%. For a middle layer,
        # it is the self.error * self.weight for each link back from a later layer to
        # this perceptron.
        self.error = (expectedOutput - self.output) * (expectedOutput - self.output)
        self.bias += self.learningRate * (expectedOutput - self.bias)
        for i in range(len(self.weights)):
            self.weights[i] += self.learningRate * (expectedOutput - self.output)


Next let's create a small network with 3 color inputs (red, green blue), 4 middle layer perceptrons, and 1 output perceptron for each color.

In [17]:
colorMap = {
    'white': (1.0, 1.0, 1.0),
    'black': (0.0, 0.0, 0.0),
    'gray': (0.5, 0.5, 0.5)
}

numMiddleLayerPerceptrons = 4
middleLayer = [ Perceptron(3) for i in range(numMiddleLayerPerceptrons) ]
outputLayer = [ Perceptron(numMiddleLayerPerceptrons) for i in range(len(colorMap))]
displayWidth = max(numMiddleLayerPerceptrons, len(colorMap)) * (middleLayer[0].printWidth + 3)
def getCentered(text, c=' '): return centerText(displayWidth, text, c)

That network looks like this:

![ColorMatchingNetDiagram2.png](./ColorMatchingNetDiagram2.png)

Finally let's create some code that will let us watch, at whatever speed we want, a color input propagate forward, then the error correction propagate backward.

In [18]:
def showMeTheNetwork(color=None, stepPauseSec=2.0):
    global middleLayer, finalLayer
    if color:
        rgb = colorMap[color]
        oneHotOutputArray = [0.0] * len(colorMap)
        i = 0
        for colorName in colorMap:
            if colorName == color:
                oneHotOutputArray[i] = 1.0
                break
            i += 1
        print("rgb:     " + getCentered(" ".join([ ("% .2f" % c) for c in rgb ])))
    else:
        print()
    print()
    print("         " + getCentered(" middle layer ", '-'))
    if color:
        for p in middleLayer: p.currentInputs = rgb
        print("inputs:  " + getCentered(" | ".join([ p.getInputsDisplay() for p in middleLayer ])))
        time.sleep(stepPauseSec)
    else:
        print()
    print("weights: " + getCentered(" | ".join([ p.getWeightsDisplay() for p in middleLayer ])))
    if color:
        time.sleep(stepPauseSec)
        for p in middleLayer:
            p.acceptInputsCheckIfTriggered(rgb)
        print("trigger?  " + " | ".join([ p.getBiasCheckDisplay() for p in middleLayer ]))
        time.sleep(stepPauseSec)
    else:
        print()
    print("outputs:  " + " | ".join([ p.getOutputDisplay() for p in middleLayer ]))
    print("         " + getCentered("", '-'))

    print()
    print("         " + getCentered(" output layer ", '-'))
    if color:
        middleLayerOutputs = []
        for p in middleLayer: middleLayerOutputs.append(p.output)
        for p in outputLayer: p.currentInputs = middleLayerOutputs
        print("inputs:  " + getCentered(" | ".join([ p.getInputsDisplay() for p in outputLayer ])))
        time.sleep(stepPauseSec)
    else:
        print()
    print("weights: " + getCentered(" | ".join([ p.getWeightsDisplay() for p in outputLayer ])))
    if color:
        time.sleep(stepPauseSec)
        for p in outputLayer:
            p.acceptInputsCheckIfTriggered(middleLayerOutputs)
        print("trigger?   " + " | ".join([ p.getBiasCheckDisplay() for p in outputLayer ]))
        time.sleep(stepPauseSec)
    else:
        print()
    print("outputs:   " + " | ".join([ p.getOutputDisplay() for p in outputLayer ]))
    print("         " + getCentered("", '-'))

    if color:
        time.sleep(stepPauseSec)
        print("expect:  " + getCentered("   ".join([ outputLayer[0].getValueDisplay(c) for c in oneHotOutputArray ])))
        time.sleep(stepPauseSec)
        print()
        print("         " + getCentered("BACKPROPAGATE - New weights and biases:"))
        for i in range(len(outputLayer)):
            outputLayer[i].backPropagate(oneHotOutputArray[i])
        time.sleep(stepPauseSec)
        print()
        print("         " + getCentered(" output layer ", '-'))
        print("weights: " + getCentered(" | ".join([ red(p.getWeightsDisplay()) for p in outputLayer ])))
        print("bias:    " + getCentered(" | ".join([ red(p.getBiasDisplay()) for p in outputLayer ])))
        
        for i in range(len(middleLayer)):
            expectedOutput = 0.0
            for j in range(len(outputLayer)):
                expectedOutput += outputLayer[j].weights[i] * outputLayer[j].error
            expectedOutput /= float(len(outputLayer))
            middleLayer[i].backPropagate(expectedOutput)
        time.sleep(stepPauseSec)
        print()
        print("         " + getCentered(" middle layer ", '-'))
        print("weights: " + getCentered(" | ".join([ red(p.getWeightsDisplay()) for p in middleLayer ])))
        print("bias:    " + getCentered(" | ".join([ red(p.getBiasDisplay()) for p in middleLayer ])))

The set of controls below let you choose which color to train next, and watch the perceptrons propagate forward then backward. Use the dropdown list to select a color to train, and cycle through the colors several times to watch the weights change, particularly when there is a difference from the actual and expected outputs.

In [12]:
interact(showMeTheNetwork, color=['white', 'gray', 'black'], stepPauseSec=(0.0, 20.0, 0.5, 3.0))

interactive(children=(Dropdown(description='color', options=('white', 'gray', 'black'), value=None), FloatSlid…

<function __main__.showMeTheNetwork(color=None, stepPauseSec=2.0)>

### Coming up...
We'll use the largest crayon box we can, and finish things up... for now.